In [1]:
!pip install xpander-sdk python-dotenv openai --force-reinstall

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached openai-1.61.0-py3-none-any.whl.metadata (27 kB)
  Using cached jsii-1.106.0-py3-none-any.whl.metadata (79 kB)
  Using cached publication-0.0.3-py2.py3-none-any.whl.metadata (8.7 kB)
  Using cached typeguard-4.2.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using c

In [2]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

In [5]:
from openai import OpenAI
from xpander_sdk import XpanderClient
xpander_client = XpanderClient(api_key=XPANDER_API_KEY)
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)

xpander_agent.add_task("Summarize the profile of david twizer from xpander.ai")
openai_client = OpenAI(api_key=OPENAI_API_KEY)
step = 1
while not xpander_agent.is_finished():
    print("-"*100)
    print(f"Step {step}")
    response = openai_client.chat.completions.create(
                model= 'o3-mini-2025-01-31',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(),
                reasoning_effort="low",
                tool_choice="auto")
            
    xpander_agent.memory.add_messages(response.model_dump())
    print("model response", response.model_dump())
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    for tool_call in tool_calls:
        print("tool name: ", tool_call.name)
        print("tool call generated payload: ", tool_call.payload)
    tool_responses = xpander_agent.run_tools(tool_calls=tool_calls)
    for tool_response in tool_responses:
        print("tool name: ", tool_response.function_name)
        print("tool status code: ", tool_response.status_code)
        print("tool result: ", tool_response.result)
    step += 1

----------------------------------------------------------------------------------------------------
Step 1
model response {'id': 'chatcmpl-AwrFGucEcttrIrN8jsOSnjiO9gEZK', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_4Sl9Wy5meOlHCIU7qmdNAxMH', 'function': {'arguments': '{"bodyParams": {},"queryParams": {"keywords": "david twizer xpander.ai"}, "pathParams": {}}', 'name': 'LinkedInProfileServiceSearchProfilesByCriteria'}, 'type': 'function'}]}}], 'created': 1738591062, 'model': 'o3-mini-2025-01-31', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_8bcaa0ca21', 'usage': {'completion_tokens': 305, 'prompt_tokens': 2216, 'total_tokens': 2521, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_det

In [6]:
print(xpander_agent.retrieve_execution_result().result)

David Twizer is the Co-Founder & CEO at xpander.ai and has a rich background in cloud solutions. Based in Tel Aviv-Yafo, Israel, he previously held prominent roles at Amazon Web Services where he served as a Senior Manager and Principal Solutions Architect, among other positions. His strong expertise in cloud computing, system architecture, and IT operations is bolstered by certifications like AWS Certified Solutions Architect and AWS Certified SysOps Administrator. David’s educational background includes studies in Computer Science at The College of Management Academic Studies and a Bachelor of Business Administration from Ruppin Academic Center. His career trajectory also includes significant experience at the Israel Defense Forces and Comm-IT, highlighting his versatile skill set and leadership in tech innovation.
